# Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.colab import files

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Training

In [ ]:
file1 = open('/content/drive/MyDrive/Kaggle/final_keywords.txt', 'r')
Lines = file1.readlines()
Data = []

for line in Lines:
    line = line.split()
    Data.append(line)

In [ ]:
Data[0]

['unit', 'enlargement', 'similarity', 'junior', 'high', 'level']

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec(sentences=Data, size=20, window=5, min_count=1, workers=4, iter=7)
model.save("word2vec.model")

In [ ]:
# import these modules
from nltk import download
download('wordnet')
download('omw-1.4')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
print(lemmatizer.lemmatize("triangles"))
print(lemmatizer.lemmatize("circumscribed"))

triangle
circumscribed


In [ ]:
model = Word2Vec.load("word2vec.model")
# Checking similarity of two words
w1 = model.wv.similarity('special', 'relativity')
w2 = model.wv.similarity('fraction', 'landmark')
w3 = model.wv.similarity('number', 'number')
w4 = model.wv.similarity('addition', 'subtraction')
w5 = model.wv.similarity('trigonometric', 'algebra')
w6 = model.wv.similarity('trigonometry', 'algebra')
# Checking similarity of two sentences
s1 = model.wv.n_similarity(['triangle', 'three', 'angle'], ['square', 'four', 'side'])
s2 = model.wv.n_similarity(['incline', 'plane', 'momentum'], ['listen', 'speak', 'english'])
s3 = model.wv.n_similarity(['trigonometric', 'algebra', 'number'], ['cell', 'biology', 'read'])
s4 = model.wv.n_similarity(['trigonometry', 'algebra', 'number'], ['cell', 'biology', 'math'])
s5 = model.wv.n_similarity(['number', 'algebra', 'addition'], ['positive', 'negative', 'fraction'])

print(w1, w2, w3, w4, w5, w6)
print(s1, s2, s3, s4, s5)

0.6252267 0.01563302 1.0 0.9690129 0.7018216 0.7256734
0.78438985 -0.16456425 0.1708943 0.41433692 0.75276726


In [ ]:
word = ['square', 'triangle', 'trigonometry']
sims = model.wv.most_similar(word, topn=10)
sims

[('polygon', 0.9078943729400635),
 ('simlar', 0.8990073800086975),
 ('quadrilateral', 0.8203004002571106),
 ('pascal', 0.7919774055480957),
 ('similarity', 0.7888847589492798),
 ('circle', 0.7697314023971558),
 ('proof', 0.7499272227287292),
 ('circumscribed', 0.7498305439949036),
 ('pythagorean', 0.74430251121521),
 ('theorem', 0.7425457239151001)]

# Model pipeline

In [ ]:
from nltk import download
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

download('stopwords')
download('punkt')
download('wordnet')
download('omw-1.4')
stop_words = set(stopwords.words('english'))
#tokenizer = RegexpTokenizer(r'\w+')
tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
lemmatizer = WordNetLemmatizer()

def clean(text):
    text = text.lower()
    word_tokens = tokenizer.tokenize(" ".join(word_tokenize(text)))

    filtered_sentence = [lemmatizer.lemmatize(w) for w in word_tokens if not w in stop_words]
    return filtered_sentence

In [ ]:
# Given topic give content
topics = [] # type text
contents =  [] # type text
similarities = []
pairs = []

for topic in topics:
    highest_indexes = [0, 0, 0, 0, 0]
    topic  = clean(topic) # nltk cleaned
    for content in contents:
        content = clean(content)
        similarity = model.wv.n_similarity(topic, content)
        similarities.append(similarity)
        for index in highest_indexes:
            if similarity > similarities[index]:
                index = contents.index(content)
                break

    temp = ''

    for index in highest_indexes:
        temp += str(contents[index])


    pairs.append([topic, temp])

In [ ]:
pairs = np.array([["sample1", "content1 content2 "], ["sample2", "content3 content4"], ["sample3", "content5 content6"]])

topics = pairs[:, 0]
contents = pairs[:, 1]

submission = pd.DataFrame.from_dict({
    'topic_id': topics,
    'content_ids': contents,
})

submission.to_csv('submission.csv')

['content1 \n content2 \n' 'content3 \n content4 \n'
 'content5 \n content6 \n']


# Accuracy testing (unfinished)

In [ ]:
### Generate pairs but in id form
id_pairs = []
topics_id = [] # type id. with similar order as 
contents_id = [] # type id with similar order as

for pair in pairs:
    topic_id = topics_id[topics.index(pair[0])]
    content_id = contents_id[contents.index(pair[1])]
    id_pairs.append([topic_id, content_id])

In [ ]:
### Converting correlations into array
correlations = []
corr = pd.read_csv('/content/drive/MyDrive/Kaggle/correlations.csv')
topic_ids = corr.topic_id.values.tolist()
content_ids = corr.content_ids.values.tolist()


for i in range(len(topic_ids)):
    correlations.append([topic_ids[i], content_ids[i]])

print(correlations)

['c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c_76231f9d0b5e', 'c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c_ebb7fdf10a7e', 'c_11a1dc0bfb99', 'c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c_b972646631cb c_d7a0d7eaf799', 'c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4', 'c_7ff92a954a3d c_8790b074383e', 'c_07f1d0eec4b2 c_15a6fb858696 c_175e9db3fc44 c_1c2e804fa58a c_1ec97b588bce c_247c609418f3 c_3a2bf4a358da c_451671e513d2 c_4698dc0a94dc c_4aea397ae55e c_5299d6442ed8 c_6e5ae1f2bb90 c_76a17cfb9d87 c_7868f655c31e c_7cc189e7acb0 c_94e49adc276f c_a035176449cd c_acaa5b1ab542 c_b8d730238789 c_ba964dd99e10 c_bc34d05f7f0a c_c1db2bde6960 c_d26e23e98356 c_d9087b07e2fc c_eef121f3ef45 c_feda60d7d9ee', 'c_2bbc650030f4 c_304ee4f59410', 'c_005e793bd0c5 c_066737fa5146 c_08b2070f92e0 c_0a0f0cfcd01a c_0e3353b058f0 c_0eda77684335 c_0f16d374f415 c_10ca1869c758 c_10dbed87d839 c_11181d0b95aa c_115c5b863574 c_1cd9f807f8e5 c_1f20565b1381 c_222e318d6d8a c_29633da013a9 c_2cd238e941f1 c_30b3480d92f8 c_34bc70d326d5

In [ ]:
correct = 0
wrong = 0

for pair in pairs:
    if pair[0] in correlations: # problem how to make it so that we only check answer
        if pair[1] in correlations[correlations.index(pair[0])][1]:
            correct += 1
        else:
            wrong += 1
        

print("accuracy is" + str( (correct)/(correct+wrong) ) + "%")

In [ ]:
"kesehotum" in "aaron is a namaste hi kesehotum"

True